# Neural Amp Modeler - Model Training Notebook
🔶**Before you run**🔶

Make sure to get a GPU! (From the upper-left menu, click Runtime->Change runtime type->Select "GPU" from the "Hardware accelerator dropdown menu)

## Step 1: Get data
* **Download the reamp signal.** Here: [input.wav](https://drive.google.com/file/d/1KbaS4oXXNEuh2aCPLwKrPdf5KFOjda8G/view?usp=drive_link).
* **Reamp your gear.** Then reamp the gear you want to model using it. Save that reamp as **"output.wav"**. *Note: Use 48kHz, 24-bit, mono.* For other sample rates, use [the CLI trainer](https://github.com/sdatkinson/neural-amp-modeler).
* **Upload your files.** Upload `input.wav` and your `output.wav` by clicking the Folder icon on the left ⬅ and then clicking the upload icon or by dragging the files into the panel.

## Step 2: Train!
Configure your training run below, then hit the Play button to start training!

🕙NOTE: At default settings, training will take about 10 minutes.🕙

In [ ]:
# Hi there! This is the code that makes this notebook work. Feel free to mess around
# with it :)

try:
    import nam
except ImportError as e:
    print("Installing NAM into Colab. This should take a couple seconds.")
    # Check what we're starting with (Issue 399)
    !if [ ! -d logs ]; then mkdir logs; fi
    !pip list > logs/packages.log
    # !pip install neural-amp-modeler==0.11 >& logs/install.log
    # Hotfix
    !pip install git+https://github.com/sdatkinson/neural-amp-modeler.git@hotfix-0.11.1 >& logs/install.log
    # Hint: use the next line instead for the very latest!
    # !pip install git+https://github.com/sdatkinson/neural-amp-modeler.git@main

from typing import Optional

from nam.train.colab import run
from nam.models.metadata import GearType, ToneType, UserMetadata

%load_ext tensorboard

from functools import partial

import ipywidgets as widgets

# NOTE: Enums need to be handled carefully since the values need to be supplied literally here!

#@markdown # Training parameters
epochs = 100 #@param {type: "number"}
architecture = "standard"  #@param ["standard", "lite", "feather", "nano"] {type: "string"}
latency_samples = "auto"  #@param {type: "string"}
ignore_checks = False #@param {type: "boolean"}

#@markdown # Metadata
use_metadata = False #@param {type: "boolean"}
name = "My model" #@param {type:"string"}
modeled_by = "Your name" #@param {type:"string"}
gear_make = "GearCo" #@param {type:"string"}
gear_model = "GearName" #@param {type:"string"}
gear_type = "amp" #@param ["amp", "pedal", "pedal_amp", "amp_cab", "amp_pedal_cab", "preamp", "studio"] {type:"string"}
tone_type = "clean" #@param ["clean", "overdrive", "crunch", "hi_gain", "fuzz"] {type:"string"}
# DRY: "(optional)" used below in _parse_level_dbu
reamp_default_value = "(optional)"
reamp_send_level = "(optional)" #@param {type:"string"}
reamp_return_level = "(optional)" #@param {type:"string"}
#@markdown _For more information on measuring your reamp send and return calibration levels, see [the documentation](https://neural-amp-modeler.readthedocs.io/en/stable/tutorials/calibration.html)._

# A few helper functions to make sense of the values provided above.
def _verbose_enum(E, val):
    try:
        return E(val)
    except ValueError as e:
        raise ValueError(
            str(e)
            + "\nValid choices are: "
            + ", ".join(list(x.value for x in E))
        )

def _parse_latency(ls: str):
    if ls.lower() == "auto":
        return None
    try:
        return int(ls)
    except ValueError as e:
        raise ValueError(
            f"Invalid value for latency {ls} was given. Either use 'auto' or provide "
            f"the reamp latency, in samples.\nOriginal exception:\n\n{e}"
        )

def _parse_level_dbu(value: str) -> Optional[float]:
    if value in {"", reamp_default_value}:
        return None
    try:
        return float(value)
    except ValueError as e:
        raise ValueError(
            f"Error parsing provided calibration level '{value}'. Either specify a "
            f"number or leave blank.\n\nOriginal exception: {e}"
        )

if not use_metadata:
    print("Metadata will be skipped")
    user_metadata = None
else:
    print("Parsing user-provided metadata...")
    user_metadata = UserMetadata(
        name=name,
        modeled_by=modeled_by,
        gear_make=gear_make,
        gear_model=gear_model,
        gear_type=_verbose_enum(GearType, gear_type.lower()),
        tone_type=_verbose_enum(ToneType, tone_type.lower()),
        input_level_dbu=_parse_level_dbu(reamp_send_level),
        output_level_dbu=_parse_level_dbu(reamp_return_level),
    )

%tensorboard --logdir /content/lightning_logs
run(
    epochs=epochs,
    architecture=architecture,
    ignore_checks=ignore_checks,
    user_metadata=user_metadata,
    delay=_parse_latency(latency_samples)
)

## Step 3: Check the results and download your model
We're done!

Have a look at the plot above to see how your model compares to the real gear you're modeling.
Hopefully it looks good!
Go to the file browser on the left panel ⬅ and download `model.nam` from the `exported_model` directory (you may need to hit the refresh button).

# 🎸 **ENJOY!** 🎸